In [1]:
%matplotlib inline
%load_ext autoreload
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import time
from datetime import datetime
import scipy.optimize as opt
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
plt.style.use("ggplot")
mpl.pyplot.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams["font.size"] = 13
#mpl.rcParams["font.family"] = 'Osaka'

import sys
sys.path.append('../code/')

from simulation_hash_price import *
from simulation_fixed_path import make_stats
from hash import *
from plot_path import *

## Setup

In [2]:
path = '../data/BTCdata_2018OctDec.csv'
df_oct_dec = pd.read_csv(path)
df_oct_dec = df_oct_dec.dropna()
df_oct_dec['height'].min(), df_oct_dec['height'].max()

(543835, 556458)

In [3]:
path = '../data/BTCdata_2.csv'
df_full = pd.read_csv(path)
df_full = df_full.dropna()

In [4]:
df_full = df_full[df_full['height']<=556458]
df = df_full[543835<=df_full['height']]

In [5]:
df['hashrate'].min(), df['hashrate'].max()

(31.518279011878057, 67.11306695962713)

In [7]:
6.94 * (10**(-5)) / 31.52

2.201776649746193e-06

* Let $H_{min} := H(0) = 31.52$.
* Assume a positive relation between hash rate and drift
  - more hash rate -> less probability of system's collapse -> the present value of bitcoin goes up
* Let $\mu(t)$ be period-$t$ drift and assume the following version of geometric Brownian motion:
$$
S(t+1) - S(t) = \mu(t) S(t) B(t) + \sigma S(t) \sqrt{B(t)} \epsilon (t),
$$
where $\epsilon(t)$ i.i.d.-drawn from $\mathcal{N}(0,1)$.
* We assume the following functional form:
$$
\mu(t) = \beta (H(t) - H(0)),
$$
* We tried different $\beta \in \{1.0 \times 10^{-6}, 1.5 \times 10^{-6}, 2.0 \times 10^{-6}, 2.5 \times 10^{-6}, 3.0 \times 10^{-6}\}$.
  - The drift is assumed to be (1) zero if the hash rate stays at the same level as the one on the first day of the simulation, and (2) -10%/day if the hash rate is zero when $\beta = 2.20  \times 10^{-6}$.

## Simulation

In [ ]:
# generate epsilons
num_iter = 5000

df_epsilon = pd.DataFrame()
for iter in range(num_iter):
    df_epsilon['iter_{}'.format(iter)] = np.random.normal(size=20000)
df_epsilon.to_csv('/Volumes/Data/research/BDA/simulation/sim_epsilons_{}obs.csv'.format(num_iter), index=False)    

In [6]:
path = '../data/BTCdata_presim.csv'
df = pd.read_csv(path)
df['time'] = pd.to_datetime(df['time'])
df = df.rename(columns={'blocktime': 'block_times', 'price': 'prices', 'probability of success /Eh': 'winning_rates'})

In [13]:
beta_list = np.arange(1, 3.5, 0.5)*(10**(-6))

In [10]:
beta_list = np.arange(2, 3.5, 0.5)*(10**(-6))

In [12]:
# takes ?? hours
for beta in beta_list:
    print(beta)
    print(datetime.now())
    generate_simulation_data(beta=beta, num_iter=5000, prev_data=df)
    print(datetime.now())
    generate_simulation_data_DAA0(beta=beta, num_iter=5000, prev_data=df)
    print(datetime.now())
    generate_simulation_data_asert(beta=beta, num_iter=5000, prev_data=df)
    print(datetime.now())

2e-06
2022-06-07 21:01:12.022533
2022-06-08 00:05:49.914893
2022-06-08 01:42:05.241552
2022-06-08 03:20:14.586901
2.4999999999999998e-06
2022-06-08 03:20:14.587074
2022-06-08 06:20:51.650392
2022-06-08 07:55:28.591876
2022-06-08 09:33:53.617928
3e-06
2022-06-08 09:33:53.618404
2022-06-08 12:26:20.819424
2022-06-08 14:01:19.596019
2022-06-08 15:40:12.253181


## Block statistics

In [20]:
dir_sim = '/Volumes/Data/research/BDA/simulation/'
for beta in np.arange(1, 3.5, 0.5):
    print('beta = {}'.format(beta))
    filelist = [dir_sim+'hash-price_DAA_asert_blocktime_ps0_5000obs_T=None_beta{}.csv'.format(beta),
                dir_sim+'hash-price_DAA-0_blocktime_ps0_5000obs_beta{}.csv'.format(beta),
                dir_sim+'hash-price_DAA-1_blocktime_ps0_5000obs_T=None_beta{}.csv'.format(beta),
                dir_sim+'hash-price_DAA-2_blocktime_ps0_5000obs_T=None_beta{}.csv'.format(beta)]
    df_stats = make_stats(filelist=filelist, dir_sim=dir_sim)
    print(df_stats)
    print()

beta = 1.0
         hash-price_DAA_asert_blocktime_ps0_5000obs_T=None_beta1.0.csv  \
mean                                               9.97151               
std                                               10.01071               
over60                                             1.00000               
over120                                            0.08880               
over180                                            0.00060               

         hash-price_DAA-0_blocktime_ps0_5000obs_beta1.0.csv  \
mean                                              9.971293    
std                                               9.969095    
over60                                            1.000000    
over120                                           0.065000    
over180                                           0.000400    

         hash-price_DAA-1_blocktime_ps0_5000obs_T=None_beta1.0.csv  \
mean                                             17.059890           
std                      